# HNSW
- HNSW 인덱스는 고차원 부동 벡터를 검색할 때 성능을 향상시킬 수 있는 그래프 기반 인덱싱 알고리즘
- 뛰어난 검색 정확도와 짧은 지연시간을 제공하는 반면, 계층적 그래프 구조를 유지하기 위해 높은 메모리 오버헤드 필요
- 하단 레이어에는 모든 데이터 포인트가 포함되며, 상위 레이어는 하단 레이어에서 샘플링 된 데이터 포인트의 하위 집합으로 구성

참고자료: https://milvus.io/docs/ko/v2.5.x/hnsw.md

## 작동 방식
1. 진입 지점: 검색은 그래프에서 미리 결정된 노드인 최사우이 레이어의 고정된 진입점에서 시작
2. GridSearch: 알고리즘은 쿼리 벡터에 더 이상 가까워질 수 없을 때까지 현재 레이어에서 가장 가까운 이웃으로 이동, 상위 레이어는 탐색 목적으로 사용되며, 하위 레이어에서 더 세밀한 검색을 위한 잠재적 진입점을 찾기 위한 필터 역할을 함
3. 레이어 하강: 현재 레이어에서 로컬 최소값에 도달하면 알고리즘은 미리 설정된 연결을 사용하여 하위 레이어로 내려가 grid search를 반복
4. 최종 다듬기: 이 프로세스는 최하위 레이어에 도달할 때까지 계속되며, 최종 세분화 단계에서는 가장 가까운 이웃을 식별

## 색인 만들기
- Milvus의 벡터 필드에 HNSW 인덱스를 구축하려면 add_index() 방법을 사용하여 index_type, metric_type 및 인덱스에 대한 추가 매개변수를 지정

In [3]:
from pymilvus import MilvusClient

index_params = MilvusClient.prepare_index_params()

index_params.add_index(
    field_name="vector", # Name of the vector field to be indexed
    index_type="HNSW", # Type of the index to create
    index_name="vector_index", # Name of the index to create
    metric_type="L2", # Metric type used to measure similarity
    params={
        "M": 64, # graph에서 연결할 수 있는 최대 노드 수
        "efConstruction": 100 # 인덱스 구조 연결을 위해 고려되는 최근접 이웃 수
    } # Index building params
)

## 인덱스에서 검색
- 인덱스가 구축되고 엔티티가 삽입되면 유사도 검색을 수행할 수 있음

In [5]:
search_params = {
    "params": {
        "ef": 10, # Number of neighbors to consider during the search
    }
}

client = MilvusClient(
    uri="http://localhost:19530"
)


res = client.search(
    collection_name="my_collection", # Collection name
    anns_field="vector", # Vector field name
    data=[[0.1, 0.2, 0.3, 0.4, 0.5]],  # Query vector
    limit=10,  # TopK results to return
    search_params=search_params
)


In [6]:
print(res)

data: [[]]
